In [305]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [306]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [307]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [308]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [309]:
# !fc-cache -f

In [310]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [311]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [339]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [340]:
%%capture pwd
!pwd

In [341]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('why-have-insurance-premiums-gone-up-so-much', 300, 500)

In [342]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [343]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [344]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [345]:
df = pd.read_excel("raw/RPI_Plot_EO_article_Feb_2024.xlsx")
df.columns=['time','all','dw','car']
df['time']=(df['time']+' 01').str.replace(' ','-')

In [346]:
readme, f, fc = save(df,"fig1_rpi",LOCAL)

,time,all,dw,car
0,2021-JAN-01,1.4,0.1,-2.4
1,2021-FEB-01,1.4,1.7,-5.5
2,2021-MAR-01,1.5,1.5,-7.6
3,2021-APR-01,2.9,2.3,-6.3
4,2021-MAY-01,3.3,1.9,-4.9


In [347]:
xmin='2021-01-01'
xmax='2024-01-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="% change over 12 months",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='time:T')
line1=base.mark_line(color=colors['eco-turquiose'],interpolate='basis',size=3).encode(y='all:Q')
line2=base.mark_line(color=colors['eco-light-blue'],interpolate='basis',size=3).encode(y='dw:Q')
line3=base.mark_line(color=colors['eco-dot'],interpolate='basis',size=3).encode(y='car:Q')
label1=line1.mark_text(text='All items',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.time>toDate('2023-12-01')")
label2=line2.mark_text(text=['Dwelling insurance', '& ground rent'],fontSize=12,color=colors['eco-light-blue'],align='left',dx=5).transform_filter("datum.time>toDate('2023-12-01')")
label3=line3.mark_text(text='Vehicles tax & insurance',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-12-01')")

layer1 = (
    (xaxis+line3+line2+line1+label1+label2+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 60, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_rpi



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [348]:
df=pd.DataFrame([
    {'cat':'Injury','value':30},
    {'cat':'Damage to the driver\'s vehicle','value':20},
    {'cat':'Damage to other vehicles & property','value':19},
    {'cat':'Overheads','value':17},
    {'cat':'Theft','value':4},
    {'cat':'Replacement vehicles','value':4},
    {'cat':'Uninsured drivers','value':4},
    {'cat':'Windscreens','value':2}])

In [349]:
readme, f, fc = save(df,"fig2_ins",LOCAL)

,cat,value
0,Injury,30
1,Damage to the driver's vehicle,20
2,Damage to other vehicles & property,19
3,Overheads,17
4,Theft,4


In [350]:
xmin='Injury'
xmax='Windscreens'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:N",
        sort='-y',
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            # labelAlign='left'
            labelAngle=-20,
        ),
        # scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x=alt.X('cat:N'))
bar=base.mark_bar(color=colors['eco-dot'],width=40).encode(y=alt.Y('value:Q'),color=alt.Color('cat:N',legend=None,
    scale=alt.Scale(range=[colors['eco-yellow'],colors['eco-orange'],colors['eco-red'],colors['eco-gray'],colors['eco-mid-blue'],colors['eco-light-blue'],
                           colors['eco-turquiose'],colors['eco-green']])
))
labels=bar.mark_text(fontSize=12,fill='#ffffff',dy=10).encode(text='value:N')

layer1 = (
    (xaxis+bar+labels).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 30, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=55).encode(x='x:N',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


fig2_ins



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 4

In [351]:
df = pd.read_csv("raw/wpd_datasets (2).csv",skiprows=1).dropna(how='all',axis=1)
df.columns=['y1','y2','y3','y4']
df['x']=['2018','2019','2020','2021','2022','2023 (through 02)']

In [352]:
readme, f, fc = save(df,"fig4_tcor",LOCAL)

,y1,y2,y3,y4,x
0,4050.473186,3624.605678,3141.955836,2356.466877,2018
1,4258.675079,3842.271293,3350.157729,2451.104101,2019
2,4618.296530,4107.255521,3558.359621,2611.987382,2020
3,4996.845426,4542.586751,4012.618297,2924.290221,2021
4,5526.813880,5157.728707,4618.296530,3312.302839,2022


In [353]:
xmin='2018'
xmax='2023 (through 02)'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="$",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='x:N')
line1=base.mark_bar(color=colors['eco-turquiose'],width=12,xOffset=-19).encode(y='y1:Q')
line2=base.mark_bar(color=colors['eco-light-blue'],width=12,xOffset=-6).encode(y='y2:Q')
line3=base.mark_bar(color=colors['eco-mid-blue'],width=12,xOffset=7).encode(y='y3:Q')
line4=base.mark_bar(color=colors['eco-gray'],width=12,xOffset=20).encode(y='y4:Q')
label1=line1.mark_text(text='Current year',fontSize=12,color=colors['eco-turquiose'],align='left',yOffset=-5,angle=270,xOffset=-19).transform_filter("datum.x=='2019'")
label2=line2.mark_text(text='1-3 years old',fontSize=12,color=colors['eco-light-blue'],align='left',yOffset=-5,angle=270,xOffset=-6).transform_filter("datum.x=='2019'")
label3=line3.mark_text(text='4-6 years old',fontSize=12,color=colors['eco-mid-blue'],align='left',yOffset=-5,angle=270,xOffset=7).transform_filter("datum.x=='2019'")
label4=line4.mark_text(text='7 years and older',fontSize=12,color=colors['eco-gray'],align='left',yOffset=-5,angle=270,xOffset=20).transform_filter("datum.x=='2019'")

layer1 = (
    (xaxis+line4+line3+line2+line1+label1+label2+label3+label4).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 6000, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=80).encode(x='x:N',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


fig4_tcor



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 5

In [354]:
df = pd.read_csv("raw/wpd_datasets.csv",skiprows=1).dropna(how='all',axis=1)
df=df.drop(['X','X.1','X.2'],axis=1)
df.columns=['y1','y2','y3']
df['x']=range(2002,2022)

In [355]:
readme, f, fc = save(df,"fig5_cell",LOCAL)

,y1,y2,y3,x
0,3.989517,NaN,NaN,2002
1,NaN,NaN,0.413757,2003
2,5.001222,NaN,NaN,2004
3,6.168475,0.187840,0.708506,2005
4,4.816785,0.454454,0.637409,2006


In [356]:
xmin=2002
xmax=2021
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='x:Q')
line1=base.mark_line(color=colors['eco-turquiose'],interpolate='basis',size=3).encode(y='y1:Q')
line2=base.mark_line(color=colors['eco-light-blue'],interpolate='basis',size=3).encode(y='y2:Q')
line3=base.mark_line(color=colors['eco-dot'],interpolate='basis',size=3).encode(y='y3:Q')
label1=line1.mark_text(text='Hand-held',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.x>2020")
label2=line2.mark_text(text=['Visible manipulation','of hand-held devices'],fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=-8).transform_filter("datum.x>2020")
label3=line3.mark_text(text=['Visible headset','cell phone use'],fontSize=12,color=colors['eco-dot'],align='left',dx=5,dy=-10).transform_filter("datum.x>2020")

layer1 = (
    (xaxis+line3+line2+line1+label1+label2+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 6.5, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=20).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig5_cell



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 6

In [357]:
df = pd.read_csv("raw/wpd_datasets3.csv",skiprows=1)
df=df.drop(['X.1','X.2'],axis=1)
df.columns=['x','y1','y2','y3']

In [358]:
readme, f, fc = save(df,"fig6_theft",LOCAL)

,x,y1,y2,y3
0,2018 Q1,1.003642,1.003642,1.187282
1,2018 Q2,0.887270,1.005339,1.188978
2,2018 Q3,1.046378,1.046257,1.177399
3,2018 Q4,0.943484,0.956477,1.127002
4,2019 Q1,0.958375,1.023704,1.089517


In [359]:
xmin='2018 Q1'
xmax='2023 Q3'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:N",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            labelExpr="indexof(datum.value,'2018 Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2020 Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2022 Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2023 Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2019 Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2021 Q1')>-1?slice(datum.value,0,4):''"
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.1f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='x:N')
line1=base.mark_line(color=colors['eco-turquiose'],interpolate='basis',size=3).encode(y='y1:Q')
line2=base.mark_line(color=colors['eco-light-blue'],interpolate='basis',size=3).encode(y='y2:Q')
line3=base.mark_line(color=colors['eco-dot'],interpolate='basis',size=3).encode(y='y3:Q')
label1=line1.mark_text(text='Asian',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.x=='2023 Q3'")
label2=line2.mark_text(text=['Domestic (US)'],fontSize=12,color=colors['eco-light-blue'],align='left',dx=5).transform_filter("datum.x=='2023 Q3'")
label3=line3.mark_text(text=['European'],fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.x=='2023 Q3'")

layer1 = (
    (xaxis+line3+line2+line1+label1+label2+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 4, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=20).encode(x='x:N',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
# layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6_theft



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 7

In [360]:
df = pd.read_csv("raw/wpd_datasets4.csv",skiprows=1)

In [361]:
readme, f, fc = save(df,"fig7_clim",LOCAL)

,X,Y
0,2012.040323,11.384379
1,2012.947581,10.065344
2,2013.975806,10.088296
3,2014.993952,10.780665
4,2015.981855,15.267002


In [362]:
xmin=2012
xmax=2022
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1,clip=True).encode(
    alt.X(
        "x:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
        scale=alt.Scale(domain=[2011.5,2022.5],nice=False),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Number of billion-dollar climate disasters",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='X:Q')
line1=base.mark_bar(color=colors['eco-turquiose'],size=20,clip=True).encode(y='Y:Q')
label1=line1.mark_text(text='Asian',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.x=='2023 Q3'")

layer1 = (
    (xaxis+line1+label1).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 24, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=20).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]
WARN Infinite extent for field "Y": [Infinity, -Infinity]


fig7_clim



alt.LayerChart(...)

alt.LayerChart(...)

# Post-process

## Clean up `SVG`s
`base64 URI` encode images

In [363]:
from base64 import b64encode

In [364]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [365]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

## README

In [366]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_rpi.csv)  
GitHub: [fig1_rpi](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/why-have-insurance-premiums-gone-up-so-much)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_rpi_local.png) [`svg`](visualisation/fig1_rpi_local.svg) [`json`](visualisation/fig1_rpi_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_rpi.png) [`svg`](visualisation/fig1_rpi.svg) [`json`](visualisation/fig1_rpi.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_rpi_local_no_branding.png) [`svg`](visualisation/fig1_rpi_local_no_branding.svg) [`json`](visualisation/fig1_rpi_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_rpi_no_branding.png) [`svg`](visualisation/fig1_rpi_no_branding.svg) [`json`](visualisation/fig1_rpi_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_rpi_local_dark.png) [`svg`](visualisation/fig1_rpi_local_dark.svg) [`json`](visualisation/fig1_rpi_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_rpi_dark.png) [`svg`](visualisation/fig1_rpi_dark.svg) [`json`](visualisation/fig1_rpi_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_rpi_local_no_branding_dark.png) [`svg`](visualisation/fig1_rpi_local_no_branding_dark.svg) [`json`](visualisation/fig1_rpi_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_rpi_no_branding_dark.png) [`svg`](visualisation/fig1_rpi_no_branding_dark.svg) [`json`](visualisation/fig1_rpi_no_branding_dark.json)   

!["fig1_rpi"](visualisation/fig1_rpi.svg "fig1_rpi")

  

!["fig1_rpi_dark"](visualisation/fig1_rpi_dark.svg "fig1_rpi")

## Figure 2  

Data: [`csv`](data/fig2_ins.csv)  
GitHub: [fig2_ins](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/why-have-insurance-premiums-gone-up-so-much)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ins_local.png) [`svg`](visualisation/fig2_ins_local.svg) [`json`](visualisation/fig2_ins_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_ins.png) [`svg`](visualisation/fig2_ins.svg) [`json`](visualisation/fig2_ins.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ins_local_no_branding.png) [`svg`](visualisation/fig2_ins_local_no_branding.svg) [`json`](visualisation/fig2_ins_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_ins_no_branding.png) [`svg`](visualisation/fig2_ins_no_branding.svg) [`json`](visualisation/fig2_ins_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ins_local_dark.png) [`svg`](visualisation/fig2_ins_local_dark.svg) [`json`](visualisation/fig2_ins_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_ins_dark.png) [`svg`](visualisation/fig2_ins_dark.svg) [`json`](visualisation/fig2_ins_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_ins_local_no_branding_dark.png) [`svg`](visualisation/fig2_ins_local_no_branding_dark.svg) [`json`](visualisation/fig2_ins_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_ins_no_branding_dark.png) [`svg`](visualisation/fig2_ins_no_branding_dark.svg) [`json`](visualisation/fig2_ins_no_branding_dark.json)   

!["fig2_ins"](visualisation/fig2_ins.svg "fig2_ins")

  

!["fig2_ins_dark"](visualisation/fig2_ins_dark.svg "fig2_ins")

## Figure 4  

Data: [`csv`](data/fig4_tcor.csv)  
GitHub: [fig4_tcor](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/why-have-insurance-premiums-gone-up-so-much)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_tcor_local.png) [`svg`](visualisation/fig4_tcor_local.svg) [`json`](visualisation/fig4_tcor_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_tcor.png) [`svg`](visualisation/fig4_tcor.svg) [`json`](visualisation/fig4_tcor.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_tcor_local_no_branding.png) [`svg`](visualisation/fig4_tcor_local_no_branding.svg) [`json`](visualisation/fig4_tcor_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_tcor_no_branding.png) [`svg`](visualisation/fig4_tcor_no_branding.svg) [`json`](visualisation/fig4_tcor_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_tcor_local_dark.png) [`svg`](visualisation/fig4_tcor_local_dark.svg) [`json`](visualisation/fig4_tcor_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_tcor_dark.png) [`svg`](visualisation/fig4_tcor_dark.svg) [`json`](visualisation/fig4_tcor_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_tcor_local_no_branding_dark.png) [`svg`](visualisation/fig4_tcor_local_no_branding_dark.svg) [`json`](visualisation/fig4_tcor_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_tcor_no_branding_dark.png) [`svg`](visualisation/fig4_tcor_no_branding_dark.svg) [`json`](visualisation/fig4_tcor_no_branding_dark.json)   

!["fig4_tcor"](visualisation/fig4_tcor.svg "fig4_tcor")

  

!["fig4_tcor_dark"](visualisation/fig4_tcor_dark.svg "fig4_tcor")

## Figure 5  

Data: [`csv`](data/fig5_cell.csv)  
GitHub: [fig5_cell](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/why-have-insurance-premiums-gone-up-so-much)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_cell_local.png) [`svg`](visualisation/fig5_cell_local.svg) [`json`](visualisation/fig5_cell_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_cell.png) [`svg`](visualisation/fig5_cell.svg) [`json`](visualisation/fig5_cell.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_cell_local_no_branding.png) [`svg`](visualisation/fig5_cell_local_no_branding.svg) [`json`](visualisation/fig5_cell_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_cell_no_branding.png) [`svg`](visualisation/fig5_cell_no_branding.svg) [`json`](visualisation/fig5_cell_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_cell_local_dark.png) [`svg`](visualisation/fig5_cell_local_dark.svg) [`json`](visualisation/fig5_cell_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_cell_dark.png) [`svg`](visualisation/fig5_cell_dark.svg) [`json`](visualisation/fig5_cell_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_cell_local_no_branding_dark.png) [`svg`](visualisation/fig5_cell_local_no_branding_dark.svg) [`json`](visualisation/fig5_cell_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_cell_no_branding_dark.png) [`svg`](visualisation/fig5_cell_no_branding_dark.svg) [`json`](visualisation/fig5_cell_no_branding_dark.json)   

!["fig5_cell"](visualisation/fig5_cell.svg "fig5_cell")

  

!["fig5_cell_dark"](visualisation/fig5_cell_dark.svg "fig5_cell")

## Figure 6  

Data: [`csv`](data/fig6_theft.csv)  
GitHub: [fig6_theft](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/why-have-insurance-premiums-gone-up-so-much)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_theft_local.png) [`svg`](visualisation/fig6_theft_local.svg) [`json`](visualisation/fig6_theft_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_theft.png) [`svg`](visualisation/fig6_theft.svg) [`json`](visualisation/fig6_theft.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_theft_local_no_branding.png) [`svg`](visualisation/fig6_theft_local_no_branding.svg) [`json`](visualisation/fig6_theft_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_theft_no_branding.png) [`svg`](visualisation/fig6_theft_no_branding.svg) [`json`](visualisation/fig6_theft_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_theft_local_dark.png) [`svg`](visualisation/fig6_theft_local_dark.svg) [`json`](visualisation/fig6_theft_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_theft_dark.png) [`svg`](visualisation/fig6_theft_dark.svg) [`json`](visualisation/fig6_theft_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_theft_local_no_branding_dark.png) [`svg`](visualisation/fig6_theft_local_no_branding_dark.svg) [`json`](visualisation/fig6_theft_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_theft_no_branding_dark.png) [`svg`](visualisation/fig6_theft_no_branding_dark.svg) [`json`](visualisation/fig6_theft_no_branding_dark.json)   

!["fig6_theft"](visualisation/fig6_theft.svg "fig6_theft")

  

!["fig6_theft_dark"](visualisation/fig6_theft_dark.svg "fig6_theft")

## Figure 7  

Data: [`csv`](data/fig7_clim.csv)  
GitHub: [fig7_clim](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/why-have-insurance-premiums-gone-up-so-much)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_clim_local.png) [`svg`](visualisation/fig7_clim_local.svg) [`json`](visualisation/fig7_clim_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7_clim.png) [`svg`](visualisation/fig7_clim.svg) [`json`](visualisation/fig7_clim.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_clim_local_no_branding.png) [`svg`](visualisation/fig7_clim_local_no_branding.svg) [`json`](visualisation/fig7_clim_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7_clim_no_branding.png) [`svg`](visualisation/fig7_clim_no_branding.svg) [`json`](visualisation/fig7_clim_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_clim_local_dark.png) [`svg`](visualisation/fig7_clim_local_dark.svg) [`json`](visualisation/fig7_clim_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7_clim_dark.png) [`svg`](visualisation/fig7_clim_dark.svg) [`json`](visualisation/fig7_clim_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_clim_local_no_branding_dark.png) [`svg`](visualisation/fig7_clim_local_no_branding_dark.svg) [`json`](visualisation/fig7_clim_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7_clim_no_branding_dark.png) [`svg`](visualisation/fig7_clim_no_branding_dark.svg) [`json`](visualisation/fig7_clim_no_branding_dark.json)   

!["fig7_clim"](visualisation/fig7_clim.svg "fig7_clim")

  

!["fig7_clim_dark"](visualisation/fig7_clim_dark.svg "fig7_clim")

